In [ ]:
import sys
sys.path.append('../../../src/')

import os
from typing import Dict
from os import PathLike
from pathlib import Path

import data_io

from harp.reader import create_reader

from utils import parse, processing, plotting_utils as plotting, AddExtraColumns

# Plotting libraries
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.backends.backend_pdf import PdfPages
from tkinter import font
from matplotlib.gridspec import GridSpec

from matplotlib.ticker import FuncFormatter, MaxNLocator, FixedLocator
import seaborn as sns
import pandas as pd
import numpy as np
import datetime

def format_func(value, tick_number):
    return f"{value:.0f}"

from numpy.typing import ArrayLike
from typing import Literal, Tuple

sns.set_context('talk')

import warnings
pd.options.mode.chained_assignment = None  # Ignore SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [ ]:
cumulative_df = pd.DataFrame(columns=['mouse', 'session', 'total_sites', 'rewarded_stops', 'unrewarded_stops', 'water', 'distance', 'session_n'])
cumulative_df_trials = pd.DataFrame()
# for mouse in ["690164","690165","690167","699894","699895","699899", "694569"]:
for mouse in ["716455"]:

    base_path = 'Z:/scratch/vr-foraging/data/'

    session_n = 0
    current_session = ''
    for file_name in os.listdir(os.path.join(base_path, mouse)):

        session_path = os.path.join(base_path, mouse, file_name)
        session = file_name[:8]
        if current_session != session:
            session_n+=1
            current_session = session
        session_path = Path(session_path)

        try:
            data = parse.load_session_data(session_path)
        except:
            print('This session is missing data: ', session)
            continue

        print(session)
        try:
            data = parse.load_session_data(session_path)
        except:
            print('Error in loading')
            continue
                
        try:
            data['harp_olfactometer'].streams.OdorValveState.load_from_file()
            data['harp_olfactometer'].streams.EndValveState.load_from_file()
        except:
            pass
        
        try:
            data['harp_behavior'].streams.OutputSet.load_from_file()
            data['harp_behavior'].streams.OutputClear.load_from_file()
            data['config'].streams['TaskLogic'].load_from_file()
        except:
            continue
        
        try:
            reward_sites, active_site, encoder_data, config = analysis.parse_data(data)
        except:
            print('Error parsing data')
            continue
            
        
        collected_df = reward_sites.loc[reward_sites['has_choice']==True].groupby(['collected','odor_label'])['reward_delivered'].count().reset_index()

        unrewarded_stops = collected_df.loc[collected_df.collected==0]['reward_delivered'].sum()
        rewarded_stops = collected_df.loc[collected_df.collected==1]['reward_delivered'].sum()
        water_collected = reward_sites.loc[(reward_sites['collected']==1)]['reward_delivered'].sum()
        total_stops = reward_sites.loc[(reward_sites['has_choice']==True)]['reward_available'].count()

        stopped_df = reward_sites.loc[(reward_sites['has_choice']==True)].groupby(['collected','odor_label'])[['reward_delivered']].sum().reset_index()
        
        print('Total sites: ' ,len(reward_sites), ' | ', 'Total rewarded stops: ',rewarded_stops, '(',  np.round((rewarded_stops/total_stops)*100,2),'%) | ', 
            'Total unrewarded stops: ',unrewarded_stops,'(',  np.round((unrewarded_stops/total_stops)*100,2),'%) | ','Water consumed: ', water_collected, 'ul',)
        if 'startPosition' in active_site.columns:
            print('Total travelled m: ', np.round(active_site.startPosition.max()/100,2))
        else:
            print('Total travelled m: ', np.round(active_site.start_position.max(),2))
            
        for odor_label in stopped_df.loc[(stopped_df.collected==1)].odor_label.unique():
            print(odor_label, ':', stopped_df.loc[(stopped_df.odor_label == odor_label)&(stopped_df.collected==1), 'reward_delivered'].iloc[0], 'ul')

        if 'startPosition' in active_site.columns:
            stop_duration = np.round(active_site.startPosition.max()/100,2)
        else:
            stop_duration = np.round(active_site.start_position.max(),2)
        print('Total travelled m: ', np.round(active_site.start_position.max(),2))
        
        cumulative_df_trials['mouse'] = mouse
        cumulative_df_trials['session'] = session
        cumulative_df_trials = pd.concat([cumulative_df_trials, reward_sites], ignore_index=True)

        # if session not in cumulative_df.session.unique() and :
        new_row = {'mouse':mouse, 'session':session, 'total_sites':len(reward_sites), 'rewarded_stops':rewarded_stops, 'unrewarded_stops':unrewarded_stops, 
                    'water':water_collected, 'distance':stop_duration, 'session_n':session_n}
        cumulative_df.loc[len(cumulative_df)] = new_row
        
        
        break


In [ ]:
date = datetime.date.today()
date_string = "4/22/2024"
date = datetime.datetime.strptime(date_string, "%m/%d/%Y").date()

In [ ]:
mouse = '715866'

In [ ]:
session_found = False

directory = os.path.join(base_path, mouse)
files = os.listdir(os.path.join(base_path, mouse))

sorted_files = sorted(files, key=lambda x: os.path.getctime(os.path.join(directory, x)), reverse=True)

for file_name in sorted_files:
    
    if session_found == True:
        break
    
    print(file_name)
    session_path = os.path.join(base_path, mouse, file_name)
    session = file_name[:8]
    session_path = Path(session_path)
    
    if datetime.date.fromtimestamp(os.path.getctime(session_path)) != date:
        continue
    else:
        print('correct date found')
        session_found = True
    

    data = analysis.load_session_data(session_path)
    

    data['harp_olfactometer'].streams.OdorValveState.load_from_file()
    data['harp_olfactometer'].streams.EndValveState.load_from_file()
    data['software_events'].streams.ActiveSite.load_from_file()
    data['software_events'].streams.ChoiceFeedback.load_from_file()
    
    data['harp_behavior'].streams.OutputSet.load_from_file()
    data['harp_behavior'].streams.OutputClear.load_from_file()
    
    reward_sites, active_site, encoder_data, config = analysis.parse_data(data)

In [ ]:
all_epochs = pd.concat([reward_sites, active_site.loc[active_site.label != 'RewardSite']])
all_epochs.sort_index(inplace=True)

In [ ]:
active_patch = -1
total_sites = -1
for i, row in all_epochs.iterrows():
    if row['label'] == 'InterPatch':
        active_patch += 1
    if row['label'] == 'InterSite':
        total_sites += 1
    all_epochs.at[i, 'active_patch'] = active_patch
    all_epochs.at[i, 'total_sites'] = total_sites
all_epochs['total_sites'] = np.where(all_epochs['total_sites'] == -1, 0, all_epochs['total_sites'])

In [ ]:
all_epochs.head(3)

In [ ]:
reward_sites

In [ ]:
cumulative_df = pd.DataFrame(columns=['odor_0'])

In [ ]:
data['config'].streams['tasklogic_input'].load_from_file()

In [ ]:
cumulative_df.at[1, 'odor_' + str(i)] = data['config'].streams['tasklogic_input'].data['environment_statistics']['patches'][i]['label']

In [ ]:
i = 0
data['config'].streams['tasklogic_input'].data['environment_statistics']['patches'][i]
cumulative_df['odor_' + str(i)] = data['config'].streams['tasklogic_input'].data['environment_statistics']['patches'][i]['label']

In [ ]:
data['config'].streams['tasklogic_input'].data['environment_statistics']['patches'][i]